# Useful functions

In [1]:
import numpy as np
import pandas as pd 
import keras
import os
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing, model_selection
from sklearn.datasets import make_moons
from sklearn.datasets import make_circles
from sklearn import datasets
from sklearn.cluster import KMeans
from tensorflow.keras.datasets import mnist

Using TensorFlow backend.
/home/quentin/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/quentin/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/quentin/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/quentin/env/lib/python3.6/site-packages/tensorflow/python/framewo

In [2]:
def get_goodXy (X,y):
    ynew = model.predict_classes(X)
    X_good =[]
    y_good=[]
    for i in range(len(X)):
        if (ynew[i]==0 and y[i]==1) or (ynew[i]==1 and y[i]==0):
            print ("error prediction for X=%s, Predicted=%s, Real=%s"% (X[i], ynew[i], y[i]))
        else :
            X_good.append(X[i])
            y_good.append(y[i])
    return X_good,y_good        

In [3]:
def get_result_layers(model,X):
    result_layers=[]
    for i in range (len(model.layers)-1):
        hidden_layers= keras.backend.function(
                [model.layers[0].input],   
                [model.layers[i].output,] 
                )    
        result_layers.append(hidden_layers([X])[0])  
    return result_layers

In [4]:
def save_result_layers(filename,X,y,result_layers):
    f = open(filename, "w")
    for nb_X in range (len(X)):
        #my_string=""
        my_string=str(y[nb_X])+','
        for nb_layers in range (len(model.layers)-1):
            my_string+="<b>,"
            for j in range (len(result_layers[nb_layers][nb_X])):
                my_string+=str(result_layers[nb_layers][nb_X][j])+','
            my_string+="</b>,"    
        my_string=my_string [0:-1]
        my_string+='\n'
        f.write(my_string)    
    f.close()

## Iris dataset 

In [5]:
url="./iris.csv"
names = ['SepalLengthCm', 'SepalWidthCm', 
         'PetalLengthCm', 'PetalWidthCm', 
         'Species']

data = pd.read_csv(url, names=names)

#Classification binaire sur Virginica et Setosa seulement
data=data[data['Species'].isin(['Iris-virginica', 'Iris-setosa'])]

i = 8
data_to_predict = data[:i].reset_index(drop = True) 
"""
reset_index() is a method to reset index of a Data Frame. 
reset_index() method sets a list of integer ranging from 0 to length of data as index. 
"""
predict_species = data_to_predict.Species 
""" Species de la class Iris """

predict_species = np.array(predict_species) 
"""
An array object satisfying the specified requirements.
https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html
"""

prediction = np.array(data_to_predict.drop(['Species'],axis= 1))

data = data[i:].reset_index(drop = True)

X = data.drop(['Species'], axis = 1) 
"""
The drop() function is used to drop specified labels from rows or columns.
"""
X = np.array(X)
y = data['Species']

encoder = LabelEncoder()
y=encoder.fit_transform(y)


train_X, test_X, train_y, test_y = model_selection.train_test_split(X,y,test_size = 0.1, random_state = 0) 

In [6]:
# Utilisation de keras comme classifieur
# mettre sigmoid comme fonction car binaire. Attention 1 seul neurone en sortie
input_dim = len(data.columns) - 1
model = Sequential()
model.add(Dense(8, input_dim = input_dim , activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(train_X, train_y, epochs = 10, batch_size = 2)



scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/10
82/82 [==============================] - 0s 3ms/step - loss: 0.9901 - acc: 0.4390
Epoch 2/10
82/82 [==============================] - 0s 493us/step - loss: 0.6593 - acc: 0.4390
Epoch 3/10
82/82 [==============================] - 0s 434us/step - loss: 0.5813 - acc: 0.4390
Epoch 4/10
82/82 [==============================] - 0s 412us/step - loss: 0.5363 - acc: 0.4390
Epoch 5/10
82/82 [==============================] - 0s 444us/step - loss: 0.4924 - acc: 0.6463
Epoch 6/10
82/82 [==============================] - 0s 439us/step - loss: 0.4538 - acc: 0.9634
Epoch 7/10
82/82 [==============================] - 0s 407us/step - loss: 0.4193 - acc: 0.9756
Epoch 8/10
82/82 [==============================] - 0s 433us/step - loss: 0.3947 - acc: 0.9878
Epoch 9/10
82/82 [==============================] - 0s 443us/step - loss: 0.3758 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 2ms/step

acc: 100.00%


In [7]:
# Récupération seulement des bons classés
X_good,y_good=get_goodXy (train_X, train_y)

In [8]:
# Récupération des valeurs de tous les layers sauf le dernier
result_layers=get_result_layers(model,X_good)

In [9]:
# Sauvegarde du fichier
# structure :
# 0/1 = valeur de la classe
# chaque valeur de layer est entourée par un []
save_result_layers("iris_8_10_8_tmp",X_good,y_good,result_layers)
# tri du fichier
os.system ('sort iris_8_10_8_tmp > iris_8_10_8_.csv')
# effacer le fichier intermédiaire
os.system ('rm iris_8_10_8_tmp')

0

In [10]:
def get_directory_layers_from_csv(filename):
    tokens=filename.split("_")
    df = pd.read_csv(filename, sep = ',', header = None) 

    
    # creation d'un répertoire pour sauver tous les fichiers
    repertoire=filename[0:-4]
    os.makedirs(repertoire, exist_ok=True)
    string = repertoire+'/'+tokens[0]+'_'
    f=[]
    filenames=[]
    for nb_tokens in range (1,len(tokens)-1):
        name_file=string+'l'+str(nb_tokens)+'_'+tokens[nb_tokens]+'.csv'
        f.append(open(name_file, "w"))
        filenames.append(name_file)
        
        
    # sauvegarde du dataframe dans une chaîne de caracteres
    ch = df.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
    vals = [','.join(ele.split()) for ele in ch]
    
    # sauvegarde dans des fichiers spécifiques par layer
    token_layer=[]
    token_exemples=[]
    for nb_exemples in range (len(vals)):
        deb=str(df[0][nb_exemples])+','
        # 1 ligne correspond à une chaine
        s=vals[nb_exemples]
        listoftokens=re.findall(r'<b>,(.+?),</b>', s)
        nb_layers=len(listoftokens)
        
        for nb_token in range (nb_layers):
            save_token=''
            save_token=deb+str(listoftokens[nb_token])+'\n'
            
            f[nb_token].write(save_token)

    # sauvegarde d'un fichier qui contient tous les layers en une fois
    # récupération des données pour enlever les <b> et </b>
    df_all=pd.DataFrame()
    myindex=0
    for nb_columns in range(df.shape[1]):
        df[nb_columns]=df[nb_columns].astype(str)
        if (df[nb_columns]!='<b>').all() and (df[nb_columns]!='</b>').all():
            df_all[myindex]=df[nb_columns]
            myindex+=1

    # construction du nom du fichier de sauvegarde
    string = repertoire+'/'+tokens[0]+'_'
    for nb_tokens in range (1,len(tokens)-1):
        string+='l'+str(nb_tokens)+'_'+tokens[nb_tokens]+'_'
    string+='.csv'       
    # sauvegarde en .csv
    df_all.to_csv(string, sep=',', encoding='utf-8',index=False)

In [11]:
#Create a directory with a specific file for all the layers
filename="iris_8_10_8_.csv"    
get_directory_layers_from_csv(filename)   

In [12]:
#Create a directory with a specific file for all the layers
filename="iris_8_10_8_.csv"    
get_directory_layers_from_csv(filename)

## Ajouter l'export des csv pour la classe 3

Cette cellule récupère les donées de la classe 3 depuis iris.csv, enlève la colonne species, fait un predict_classes sur la classe 3, récupère les fonctions d'activation et les enregistre layer par layer dans un sous-répertoire "iris_8_10_8_C3" dans des csv nommés indentiquement à ceux pour les classes 1 et 2. On rajoute également la classe prédite en première colonne pour rester cohérent avec les autres CSV.


In [13]:
data = pd.read_csv(url, names=names)
data=data[data['Species'].isin(['Iris-versicolor'])].drop(['Species'],axis= 1)
classe3 = model.predict_classes(data)
#print(classe3)

result_layersC3=get_result_layers(model,data)
#print(result_layersC3)

#On rajoute une première colonne avec la classe prédite pour être cohérent avec les autres csv
for i,arr  in enumerate(result_layersC3):
    arr =  np.insert(arr,0,classe3[i], axis=1)
    result_layersC3[i] = arr
                                               
rep = "iris_8_10_8_C3/"
os.makedirs("iris_8_10_8_C3", exist_ok=True)
for i, (filename, arr) in enumerate(zip(["iris_l1_8.csv", "iris_l2_10.csv", "iris_l3_8.csv"],result_layersC3)):
    np.savetxt(rep+filename, arr, delimiter=",", fmt="%f")

In [14]:
def save_result_cluster(filename,clusters,numLayer,cluster_class):
#    f = open(filename, "w")
    idClusters=np.unique(clusters)
    tabCount={}
    for id in idClusters:
        count = 0
        for j in range(len(clusters)):
            if clusters[j]==id:
                tabCount[id]=count+1
                count+=1
    print(tabCount)
    my_string=""
    for key,value in tabCount.items():
        my_string += str(numLayer)+','+str(cluster_class)+','
        my_string+=str(value)+','+str(key)+'\n'
    print(my_string)
#     f.close()

def clusterizeLayer(url, nbClusters=4):
    data = pd.read_csv(url,names=names)
    dataC1=dataC2=[]
    indexC2=indexC1=[]
    i=0
    for index, row in data.iterrows():
        if row[0]==0:
            dataC1.append(row)
            indexC1=i
        else:
            dataC2.append(row)
            indexC2=i
        i+=1
        
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#         print(data)

    algo = KMeans(n_clusters = nbClusters)
    #fit_predict fait tourner l'algo et retourne y où y[i] est l'indice du cluster auquel dataC12[i] appartient
    clustersC1C2 = algo.fit_predict(data)
#     print(clustersC1C2)

    dataC3=pd.read_csv('iris.csv',names=names)
    dataC3=dataC3[dataC3['Species'].isin(['Iris-versicolor'])]
    dataC3['Species']=1.0

#     print("print dataC3:\n")
#     print(dataC3)

    #predict utilise les clusters calculés par fit_predict(dataC1C2) et attribue chaque ligne de dataC3 au cluster dont le centre
    #minimise la distance
    C3ClusterPred = algo.predict(dataC3)

#     print("Clusters C3:\n")
#     print(C3ClusterPred)
    return clustersC1C2, C3ClusterPred

## Remplacer clusterizeLayer

Problèmes: 

1. Les .csv contiennent en première colonne la classe (0 ou 1) de l'iris. Cette information ne fait pas partie des fonctions d'activation (sauf la dernière layer) et ne devrait pas être incluse dans le clustering. En outre dès qu'on veut rajouter d'autres images sans information sur la classe (en dehors de l'entrainement donc) on n'a pas d'infos sur la classe.
  Solution:
   * Nommer correctement les colonnes lors de l'importation (première colonne = "classe", et les suivantes "neuroneN")
   * Faire un drop sur la colonne classe pour ne récupérer que les colonnes correspondant aux fonctions d'activation
2. Pour la classe 3 on utilisait les données brutes. A la place je récupère les données des fonctions d'activation de la classe (avec un truc un peu moche ligne 16 pour déterminer le nom du fichier csv de la classe 3 à partir de celui des classes 1 et 2 données par le paramètre url). Pour rester cohérent avec les classes 1 et 2 le numéro de classe est aussi présent en première colonne, je l'enlève donc également.

In [15]:
def clusterizeLayer(url, nbClusters=4):
    data = pd.read_csv(url, header=None)

    data.columns = ['classe']+["neurone{}".format(i) for i in range(len(data.columns)-1)]
    
    data = data.drop(['classe'],axis= 1)
        
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#         print(data)

    algo = KMeans(n_clusters = nbClusters)
    #fit_predict fait tourner l'algo et retourne y où y[i] est l'indice du cluster auquel dataC12[i] appartient
    clustersC1C2 = algo.fit_predict(data)
#     print(clustersC1C2)

    csvname = "iris_8_10_8_C3/"+url.split("/")[2]
    dataC3=pd.read_csv(csvname, header=None)
    dataC3.columns = ['classe']+["neurone{}".format(i) for i in range(len(dataC3.columns)-1)]
    dataC3 = dataC3.drop(['classe'],axis= 1)


#     print("print dataC3:\n")
#     print(dataC3)

    #predict utilise les clusters calculés par fit_predict(dataC1C2) et attribue chaque ligne de dataC3 au cluster dont le centre
    #minimise la distance
    C3ClusterPred = algo.predict(dataC3)

#     print("Clusters C3:\n")
#     print(C3ClusterPred)

    return clustersC1C2, C3ClusterPred

In [16]:
# clustersC3l1=clusterizeLayer('./iris_8_10_8_/iris_l1_8.csv')
# clustersC3l2=clusterizeLayer('./iris_8_10_8_/iris_l2_10.csv')
# clustersC3l3=clusterizeLayer('./iris_8_10_8_/iris_l3_8.csv')

save_result_cluster("osef",clusterizeLayer('./iris_8_10_8_/iris_l1_8.csv',4)[1],1,3)
save_result_cluster("osef",clusterizeLayer('./iris_8_10_8_/iris_l2_10.csv',4)[1],2,3)
save_result_cluster("osef",clusterizeLayer('./iris_8_10_8_/iris_l3_8.csv',4)[1],3,3)

{1: 2, 2: 47, 3: 1}
1,3,2,1
1,3,47,2
1,3,1,3

{0: 1, 3: 49}
2,3,1,0
2,3,49,3

{0: 50}
3,3,50,0



In [17]:
from sklearn.decomposition import PCA
from numpy import ndarray
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt
import math


def plot_clusters_3D (X,y_predict, nb_clusters, pca_done=False):
    if pca_done==False:
        pca = PCA(n_components=3) 
        pca.fit(X) 
        pca_data = pd.DataFrame(pca.transform(X))
    else: pca_data = pd.DataFrame(X)
    colors = list(zip(*sorted(( 
                    tuple(mcolors.rgb_to_hsv( 
                          mcolors.to_rgba(color)[:3])), name) 
                     for name, color in dict( 
                            mcolors.BASE_COLORS, **mcolors.CSS4_COLORS 
                                                      ).items())))[1] 
    # number of steps to taken generate n(clusters) colors
    skips = math.floor(len(colors[5 : -5])/nb_clusters) 
    cluster_colors = colors[5 : -5 : skips] 
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(111, projection = '3d') 
    ax.scatter(pca_data[0], pca_data[1], pca_data[2],
           c = list(map(lambda label : cluster_colors[label], 
                                            y_predict))) 

    str_labels = list(map(lambda label:'% s' % label, y_predict)) 

    list(map(lambda data1, data2, data3, str_label: 
        ax.text(data1, data2, data3, s = str_label, size = 16.5, 
        zorder = 20, color = 'k'), pca_data[0], pca_data[1], 
        pca_data[2], str_labels)) 

    plt.show()
    
def plot_clusters_2D (X,y_predict, nb_clusters, pca_done=False):
    if pca_done==False:
        pca = PCA(n_components=2) 
        X_r = pca.fit(X).transform(X)
    else: X_r = X
    data = pd.DataFrame(X_r, columns=['x','y'])
    data['label']=y_predict
    list_clusters=list(set(y_predict))
    print (list_clusters)
    #create a new figure
    plt.figure(figsize=(5,5))

    #loop through labels and plot each cluster
    for i, label in enumerate(list_clusters):

        #add data points 
        plt.scatter(x=data.loc[data['label']==label, 'x'], 
                y=data.loc[data['label']==label,'y'], 
                color=color[i], 
                alpha=0.20)

        #add label
        plt.annotate(label, 
                 data.loc[data['label']==label,['x','y']].mean(),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=20, weight='bold',
                 color=color[i])
    
        

        
def get_directory_layers_from_csv(filename):
    tokens=filename.split("_")
    df = pd.read_csv(filename, sep = ',', header = None) 

    
    # creation d'un répertoire pour sauver tous les fichiers
    repertoire=filename[0:-4]
    os.makedirs(repertoire, exist_ok=True)
    string = repertoire+'/'+tokens[0]+'_'
    f=[]
    filenames=[]
    for nb_tokens in range (1,len(tokens)-1):
        name_file=string+'l'+str(nb_tokens)+'_'+tokens[nb_tokens]+'.csv'
        f.append(open(name_file, "w"))
        filenames.append(name_file)
        
        
    # sauvegarde du dataframe dans une chaîne de caracteres
    ch = df.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
    vals = [','.join(ele.split()) for ele in ch]
    
    # sauvegarde dans des fichiers spécifiques par layer
    token_layer=[]
    token_exemples=[]
    for nb_exemples in range (len(vals)):
        deb=str(df[0][nb_exemples])+','
        # 1 ligne correspond à une chaine
        s=vals[nb_exemples]
        listoftokens=re.findall(r'<b>,(.+?),</b>', s)
        nb_layers=len(listoftokens)
        
        for nb_token in range (nb_layers):
            save_token=''
            save_token=deb+str(listoftokens[nb_token])+'\n'
            
            f[nb_token].write(save_token)

    # sauvegarde d'un fichier qui contient tous les layers en une fois
    # récupération des données pour enlever les <b> et </b>
    df_all=pd.DataFrame()
    myindex=0
    for nb_columns in range(df.shape[1]):
        df[nb_columns]=df[nb_columns].astype(str)
        if (df[nb_columns]!='<b>').all() and (df[nb_columns]!='</b>').all():
            df_all[myindex]=df[nb_columns]
            myindex+=1

    # construction du nom du fichier de sauvegarde
    string = repertoire+'/'+tokens[0]+'_'
    for nb_tokens in range (1,len(tokens)-1):
        string+='l'+str(nb_tokens)+'_'+tokens[nb_tokens]+'_'
    string+='.csv'       
    # sauvegarde en .csv
    df_all.to_csv(string, sep=',', encoding='utf-8',index=False)

## Rajouter cellule pour l'affichage

In [18]:
def plot_clusters_2D_title (X,y_predict, classes, nb_clusters, title, pca_done=False):
    color = ['red', 'blue', 'purple', 'green']
    markers = ["D", "^","o"]
    if pca_done==False:
        pca = PCA(n_components=2) 
        X_r = pca.fit(X).transform(X)
    else: X_r = X
    data = pd.DataFrame(X_r, columns=['x','y'])
    data['label']=y_predict
    list_clusters=list(set(y_predict))
    #create a new figure
    plt.figure(figsize=(8,8))
    
    plt.title(title)

    #loop through clusters (for different colors)
    for i, label in enumerate(list_clusters):
        # loop for classes (for different markers)
        for j,classe in enumerate(classes.unique()):
            #add data points 
            plt.scatter(x=data.loc[(data['label']==label) & (classes==classe), 'x'], 
                    y=data.loc[(data['label']==label) & (classes==classe),'y'], 
                    s = 150,
                    color=color[i],
                    marker = markers[j],
                    alpha=0.4)

        #add cluster number
        plt.annotate(label, 
                 data.loc[data['label']==label,['x','y']].mean(),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=20, weight='bold',
                 color="k"#color[i]
                    )
    
    #add legend
    from matplotlib.lines import Line2D
    legend_elements = []
    for i,classe in enumerate(classes.unique()):
        legend_elements.append(Line2D([0], [0],
                                marker = markers[i],
                                color ="w",
                                markerfacecolor = 'gray',
                                markersize = 10,
                                markeredgewidth=0.5,
                                markeredgecolor = 'k',
                                label = "Classe {}".format(i)))
    plt.legend(handles = legend_elements)

        
def getNameC3CSV(csv):
    return"iris_8_10_8_C3/"+csv.split("/")[2]

        
def afficherClasses_2(csv, nb_clusters, nb_neurones, classinfo):
    clustersC1C2, C3ClusterPred = clusterizeLayer(csv, nb_clusters)
    data = pd.read_csv(csv, names=['classe']+list(range(nb_neurones)))
    classes = pd.Series(data['classe'])
    data = data.drop(['classe'], axis = 1)
    print(len(data))
    print(len(clustersC1C2))
    plot_clusters_2D_title_2 (data, clustersC1C2,classes, nb_clusters, csv+" "+classinfo)

def afficherClassesTogether(csv, nb_clusters, nb_neurones, classinfo):
    clustersC1C2, C3ClusterPred = clusterizeLayer(csv, nb_clusters)
    data = pd.read_csv(csv, names=['classe']+list(range(nb_neurones)))
    classes = pd.Series(data['classe'])
    dataC1C2 = data.drop(['classe'], axis = 1)
  
    dataC3 = pd.read_csv(getNameC3CSV(csv), names=['classe']+list(range(nb_neurones)))
    #classes = pd.concat([classes,pd.Series(dataC3['classe'])], ignore_index = True)
    # Le csv contient la classe prédite (soit 0 ou 1 nécessairement) mais on doit spécifier que c'est en fait la classe 2
    # On rajoute donc une colonne avec que des 2
    classe3 = pd.Series([2 for _ in range(len(dataC3))])
    classes = pd.concat([classes,classe3], ignore_index = True)

    dataC3 = dataC3.drop(['classe'], axis = 1)

    data_all = pd.concat([dataC1C2, dataC3], axis=0, ignore_index = True)
    cluster_all = np.concatenate([clustersC1C2, C3ClusterPred], axis=0)
    
    
    plot_clusters_2D_title_2 (data_all, cluster_all, classes, nb_clusters, csv+" "+"3 classes")



listeLayers = [('./iris_8_10_8_/iris_l1_8.csv', 8,  "C1 et C2"),
               ('./iris_8_10_8_/iris_l2_10.csv',10, "C1 et C2"),
               ('./iris_8_10_8_/iris_l3_8.csv',8, "C1 et C2")]
for csv, nb_neurones, classes in listeLayers:
    afficherClassesTogether(csv, 4, nb_neurones, classes)
    #afficherClasses_2(csv, 4, nb_neurones, classes)

        

NameError: name 'plot_clusters_2D_title_2' is not defined

## MNIST Dataset

In [ ]:
#(X_train, y_train), (X_test, y_test) = mnist.load_data()
from numpy import load

data = load('mnist.npz')
lst = data.files
print(lst)

(X_train,y_train) = (data[lst[1]],data[lst[2]])
(X_test,y_test) = (data[lst[0]],data[lst[3]])

X_train_sample=X_train[0:100]
y_train_sample=y_train[0:100]

X_train=X_train_sample
y_train=y_train_sample
X_train = X_train.reshape(100, 784)
X_train = X_train.astype('float32')

X_train /= 255

X_01=[]
y_01=[]
nb_X=0
for i in range(X_train.shape[0]):
    if (y_train[i]==0 or y_train[i]==1):
        
        nb_X+=1
        X_01.append(X_train[i])
        y_01.append(y_train[i])

       
train_X=np.asarray(X_01)

train_y=y_01

encoder = LabelEncoder()
train_y=encoder.fit_transform(train_y)

In [ ]:
input_dim = 784

model = Sequential()
model.add(Dense(512, input_dim = input_dim , activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(train_X, train_y, epochs = 40, batch_size = 32)


X_good,y_good=get_goodXy (train_X, train_y)


# Récupération des valeurs de tous les layers sauf le dernier
result_layers=get_result_layers(model,X_good)

In [ ]:
# Sauvegarde du fichier
# structure :
# 0/1 = valeur de la classe
# chaque valeur de layer est entourée par une étoile *
save_result_layers("mnist_512_tmp",X_good,y_good,result_layers)
# tri du fichier
os.system ('sort mnist_512_tmp > mnist_512_.csv')
# effacer le fichier intermédiaire
os.system ('rm mnist_512_tmp')

In [ ]:
normalise = True
includeClasses = True
nbClusters = 10

data = load('mnist.npz')
lst = data.files

print(data[lst[0]][0]) #X_test
print(data[lst[1]][0]) #X_train
print(data[lst[2]][0]) #y_train
print(data[lst[3]][0]) #y_test


"""
#On rajoute une colonne Classe qui contient le numéro de la classe correspondant au feature Species
listeSpecies = pd.unique(data['Species'])
listeIndices = [i+1 for i in range(len(listeSpecies))]
dico = dict(zip(listeSpecies, listeIndices))
data.insert(len(data.columns), 'Classe', [dico[x] for x in data['Species']])

if normalise:
    if includeClasses:
        m = data.columns[data.columns!='Species']
    else:
        #On veut garder la séparation en classes 1, 2 et 3, on exclut donc cette colonne de la normalisation
        m = data.columns[~data.columns.isin(['Species','Classe'])]
    data[m] = (data[m]-data[m].min())/(data[m].max()-data[m].min())

#Note: dataC3 et dataC1C2 sont des copies profondes
dataC1C2 = data[data['Species'].isin(['Iris-virginica','Iris-setosa'])].drop('Species', axis=1)
dataC3 = data[data['Species']=='Iris-versicolor'].drop('Species', axis=1)


algo = KMeans(n_clusters = nbClusters)
#fit_predict fait tourner l'algo et retourne y où y[i] est l'indice du cluster auquel dataC12[i] appartient
clustersC1C2 = algo.fit_predict(dataC1C2)

#predict utilise les clusters calculés par fit_predict(dataC12) et attribue chaque ligne de dataC3 au cluster dont le centre
#minimise la distance
C3ClusterPred = algo.predict(dataC3)

#Les Species sont-elles bien discrimninées par l'algo de clustering?
clusters = [[] for _ in range(1+max(clustersC1C2))]
for i,x in enumerate(clustersC1C2):
    clusters[x].append(i)
taillesClusters = [len(l) for l in clusters]
print("Tailles des clusters non vides")
print(taillesClusters)

for cluster in clusters:
    if not(cluster):
        continue
    classe = data.at[cluster[0],'Species']
    for sign in cluster:
        if data.at[sign, 'Species'] != classe:
            print("Présence classe mixte")

#Predict sur la classe 3
X = dataC3.drop(columns= ['Classe'])
X = np.array(X)
Y = model.predict(X)
def sigmoid(x):
    return 1/(1 + np.exp(-x))
Y = sigmoid(Y)
print(Y)
"""